In [20]:
import numpy as np
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import re
import plotly.express as px

In [2]:
def convert_to_number(val):
    if pd.isna(val):
        return np.nan
    val = str(val).replace(' ', '')  # удаляем пробелы
    # обработка значений '<число'
    if val.startswith('<'):
        num = re.findall(r'<(\d+\.?\d*)', val)
        return float(num[0]) if num else np.nan
    # обработка значений с ±
    elif '±' in val:
        nums = re.findall(r'([\d\.]+)±([\d\.]+)', val)
        if nums:
            main, uncertainty = nums[0]
            return float(main)  # берём только среднее (первое число)
        else:
            return np.nan
    # обработка значений с '/'
    elif '/' in val:
        nums = re.findall(r'([\d\.]+)/([\d\.]+)', val)
        if nums:
            num1, num2 = nums[0]
            return (float(num1) + float(num2)) / 2
        else:
            return np.nan
    # пробуем просто преобразовать в число
    else:
        try:
            return float(val)
        except:
            return np.nan

In [6]:
# Load dataset
df = pd.read_csv("for_regr_with_descrip.csv")
df['raw_efficiency'] = df['raw_efficiency'].apply(convert_to_number)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 877 entries, 0 to 876
Data columns (total 59 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    877 non-null    int64  
 1   sequence              877 non-null    object 
 2   extra_name            36 non-null     object 
 3   cpp_category          170 non-null    object 
 4   is_cpp                877 non-null    bool   
 5   cpp_type              11 non-null     object 
 6   origin                1 non-null      object 
 7   id_uptake             877 non-null    float64
 8   peptide               877 non-null    float64
 9   uptake_type           850 non-null    object 
 10  raw_efficiency        877 non-null    float64
 11  raw_toxicity          0 non-null      float64
 12  raw_concentration     0 non-null      float64
 13  id_experiment         875 non-null    float64
 14  peptide_experiment    875 non-null    float64
 15  raw_time              8

In [21]:
fig = px.box(df, x="uptake_type", y="raw_efficiency"
)

fig.show()

In [24]:
df.describe()

,id,id_uptake,peptide,raw_efficiency,raw_toxicity,raw_concentration,id_experiment,peptide_experiment,id_article,pubmed_id,...,Positive_AA,Negative_AA,MolWt,LogP,TPSA,HBD,HBA,RotBonds,Rings,Fsp3
count,877.000000,877.000000,877.000000,8.770000e+02,0.0,0.0,875.000000,875.000000,288.000000,0.0,...,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000
mean,2336.399088,858.465222,2433.510832,6.171126e+03,NaN,NaN,1155.318857,2433.202286,1.715278,NaN,...,6.605473,0.769669,2316.701556,-10.018074,1028.797423,38.883694,31.489168,77.941847,4.172178,0.619444
std,671.436748,614.014833,514.474697,5.332707e+04,NaN,NaN,458.197301,514.662395,1.553520,NaN,...,4.804808,1.707315,1806.097391,10.852696,743.684017,26.751737,24.706892,59.429384,4.624607,0.103138
min,19.000000,2.000000,1340.000000,0.000000e+00,NaN,NaN,9.000000,1340.000000,1.000000,NaN,...,0.000000,0.000000,115.176000,-167.810640,43.090000,1.000000,2.000000,1.000000,0.000000,0.300000
25%,1819.000000,279.000000,1963.000000,9.762000e+00,NaN,NaN,871.500000,1962.500000,1.000000,NaN,...,4.000000,0.000000,1451.710000,-12.835010,664.610000,25.000000,19.000000,49.000000,1.000000,0.559322
50%,2607.000000,724.000000,2607.000000,6.500000e+01,NaN,NaN,1259.000000,2607.000000,1.000000,NaN,...,6.000000,0.000000,1964.351000,-7.727470,870.180000,34.000000,26.000000,67.000000,3.000000,0.630435
75%,2847.000000,1408.000000,2847.000000,5.550000e+02,NaN,NaN,1529.500000,2846.500000,1.000000,NaN,...,8.000000,1.000000,2610.914000,-4.324860,1159.280000,45.000000,36.000000,88.000000,5.000000,0.694737
max,3103.000000,1965.000000,3103.000000,1.037500e+06,NaN,NaN,1788.000000,3103.000000,6.000000,NaN,...,42.000000,13.000000,23968.291000,6.476900,9159.580000,298.000000,306.000000,780.000000,44.000000,0.833333


# Код Кати + фильтрование по аптейку

In [25]:
def filter_data(df, arrays, cell_line_array=None, is_y=False):
    """
    Фильтрует данные по маске из столбца 'uptake_type' датафрейма df.
    
    Параметры:
    - df: pandas.DataFrame с исходными данными
    - arrays: список массивов для фильтрации и объединения (например, [X_numerical, blomap_pca])
    - cell_line_array: дополнительный массив (X_cell_line), который может отсутствовать
    - is_y: флаг, что обрабатываем целевую переменную y
    
    Возвращает:
    - Отфильтрованный массив или датафрейм
    """
    # Создаем булеву маску
    mask = df['uptake_type'].isin(['Mean Fluorescence intensity', 'Fluorescence intensity'])
    
    if is_y:
        # Для y просто применяем маску к столбцу
        return df['raw_efficiency'].values[mask]
    else:
        # Фильтруем все переданные массивы
        filtered_arrays = [arr[mask] for arr in arrays]
        
        # Объединяем массивы
        filtered_X = np.hstack(filtered_arrays)
        
        # Добавляем cell_line, если он не пуст
        if cell_line_array is not None and not cell_line_array.empty:
            filtered_cell_line = cell_line_array.values[mask]
            filtered_X = np.hstack([filtered_X, filtered_cell_line])
        
        return filtered_X

In [16]:
# Load embeddings
blomap_embeddings = np.load("blomap_regr.npy")
fingerprints_embeddings = np.load("fingerprints_regr.npy")
protbert_embeddings = np.load("protbert_regr.npy")

# Apply PCA to Blomap for XGBoost
pca_blomap = PCA(n_components=10, random_state=42)
blomap_pca = pca_blomap.fit_transform(blomap_embeddings)

# Select numerical features
selected_features = [
    "MW", "GRAVY", "pI", "Charge", "Charge_Density", "Aromaticity",
    "Flexibility", "Aliphatic_Index", "Boman_Index", "Hydrophobic_AA",
    "Polar_AA", "Positive_AA", "Negative_AA", "MolWt", "LogP",
    "TPSA", "HBD", "HBA", "RotBonds", "Rings", "Fsp3"
]
X_numerical = df[selected_features].copy()

# One-hot encoding for cell_line
if "cell_line" in df.columns:
    enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    cell_line_encoded = enc.fit_transform(df[['cell_line']])
    cell_line_feature_names = enc.get_feature_names_out(["cell_line"])
    X_cell_line = pd.DataFrame(cell_line_encoded, columns=cell_line_feature_names)
else:
    X_cell_line = pd.DataFrame()

# Prepare feature matrices
X_xgb = np.hstack([X_numerical.values, blomap_pca, fingerprints_embeddings, protbert_embeddings])
if not X_cell_line.empty:
    X_xgb = np.hstack([X_xgb, X_cell_line.values])

X_lgbm = np.hstack([X_numerical.values, blomap_embeddings, fingerprints_embeddings, protbert_embeddings])
if not X_cell_line.empty:
    X_lgbm = np.hstack([X_lgbm, X_cell_line.values])

y = df["raw_efficiency"].values

# Пример создания исходных переменных:
# (предполагается, что переменные X_numerical, blomap_pca, fingerprints_embeddings, 
#  protbert_embeddings, blomap_embeddings, X_cell_line и df уже определены)

# Для X_xgb:
# Список массивов для X_xgb
arrays_xgb = [
    X_numerical.values,
    blomap_pca,          # Из вашего исходного кода
    fingerprints_embeddings,
    protbert_embeddings
]

# Применяем фильтрацию
X_xgb = filter_data(
    df=df,
    arrays=arrays_xgb,
    cell_line_array=X_cell_line  # Передаем X_cell_line, если он не пуст
)

# Для X_lgbm:
# Список массивов для X_lgbm (здесь другой blomap)
arrays_lgbm = [
    X_numerical.values,
    blomap_embeddings,    # Разный признак относительно X_xgb
    fingerprints_embeddings,
    protbert_embeddings
]

# Применяем фильтрацию
X_lgbm = filter_data(
    df=df,
    arrays=arrays_lgbm,
    cell_line_array=X_cell_line
)

# Целевая переменная:
y = df["raw_efficiency"].values 

y = filter_data(
    df=df,
    arrays=[],  # Для y не нужны дополнительные массивы
    cell_line_array=None,
    is_y=True   # Указываем, что обрабатываем целевую переменную
)


valid_idx = ~np.isnan(y)
X_xgb, X_lgbm, y = X_xgb[valid_idx], X_lgbm[valid_idx], y[valid_idx]

# Handle missing values
imputer = SimpleImputer(strategy="mean")
X_xgb, X_lgbm = imputer.fit_transform(X_xgb), imputer.fit_transform(X_lgbm)

# Log-transform target variable
y = np.log1p(y)

# Train-test split
X_train_xgb, X_test_xgb, y_train, y_test = train_test_split(X_xgb, y, test_size=0.2, random_state=42)
X_train_lgbm, X_test_lgbm, _, _ = train_test_split(X_lgbm, y, test_size=0.2, random_state=42)

# Train XGBoost
xgb_model = XGBRegressor(n_estimators=754, max_depth=6, learning_rate=0.054886325307314195,
                         subsample=0.9967873263465272, colsample_bytree=0.8645926672674225,
                         random_state=42)
xgb_model.fit(X_train_xgb, y_train)
xgb_pred = np.expm1(xgb_model.predict(X_test_xgb))

# Train LightGBM
lgbm_model = LGBMRegressor(n_estimators=629, learning_rate=0.0114315426267485, num_leaves=77, 
                            min_data_in_leaf=9, max_depth=7, colsample_bytree=0.7, random_state=42)
lgbm_model.fit(X_train_lgbm, y_train)
lgbm_pred = np.expm1(lgbm_model.predict(X_test_lgbm))

# Ensemble predictions (90% XGBoost, 10% LightGBM)
ensemble_pred = (0.9 * xgb_pred + 0.1 * lgbm_pred)

# Evaluate model
print(f"MAE XGBoost: {mean_absolute_error(np.expm1(y_test), xgb_pred):.4f}")
print(f"MAE LightGBM: {mean_absolute_error(np.expm1(y_test), lgbm_pred):.4f}")
print(f"MAE Ensemble (90% XGBoost, 10% LightGBM): {mean_absolute_error(np.expm1(y_test), ensemble_pred):.4f}")


[LightGBM] [Warning] min_data_in_leaf is set=9, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9
[LightGBM] [Warning] min_data_in_leaf is set=9, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001706 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2205
[LightGBM] [Info] Number of data points in the train set: 249, number of used features: 326
[LightGBM] [Info] Start training from score 5.483091
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

c:\Users\ALI\miniconda3\envs\my_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [11]:
X_xgb

array([[ 3.15183620e+03, -1.49230769e+00,  1.16083223e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.78519390e+03, -1.27272727e+00,  1.18393770e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.07836580e+03, -1.12352941e+00,  1.18244848e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 8.90932000e+01,  1.80000000e+00,  5.57001667e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.71297160e+03, -3.61153846e+00,  1.19999678e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.71297160e+03, -3.61153846e+00,  1.19999678e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [17]:
import numpy as np
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Load dataset
df = pd.read_csv("for_regr_with_descrip.csv")
df['raw_efficiency'] = df['raw_efficiency'].apply(convert_to_number)

# Load embeddings
blomap_embeddings = np.load("blomap_regr.npy")
fingerprints_embeddings = np.load("fingerprints_regr.npy")
protbert_embeddings = np.load("protbert_regr.npy")

# Apply PCA to Blomap for XGBoost
pca_blomap = PCA(n_components=10, random_state=42)
blomap_pca = pca_blomap.fit_transform(blomap_embeddings)

# Select numerical features
selected_features = [
    "MW", "GRAVY", "pI", "Charge", "Charge_Density", "Aromaticity",
    "Flexibility", "Aliphatic_Index", "Boman_Index", "Hydrophobic_AA",
    "Polar_AA", "Positive_AA", "Negative_AA", "MolWt", "LogP",
    "TPSA", "HBD", "HBA", "RotBonds", "Rings", "Fsp3"
]
X_numerical = df[selected_features].copy()

# One-hot encoding for cell_line
if "cell_line" in df.columns:
    enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    cell_line_encoded = enc.fit_transform(df[['cell_line']])
    cell_line_feature_names = enc.get_feature_names_out(["cell_line"])
    X_cell_line = pd.DataFrame(cell_line_encoded, columns=cell_line_feature_names)
else:
    X_cell_line = pd.DataFrame()

# Prepare feature matrices
X_xgb = np.hstack([X_numerical, blomap_pca, fingerprints_embeddings, protbert_embeddings])
if not X_cell_line.empty:
    X_xgb = np.hstack([X_xgb, X_cell_line])

X_lgbm = np.hstack([X_numerical, blomap_embeddings, fingerprints_embeddings, protbert_embeddings])
if not X_cell_line.empty:
    X_lgbm = np.hstack([X_lgbm, X_cell_line])

y = df["raw_efficiency"].values
valid_idx = ~np.isnan(y)
X_xgb, X_lgbm, y = X_xgb[valid_idx], X_lgbm[valid_idx], y[valid_idx]

# Handle missing values
imputer = SimpleImputer(strategy="mean")
X_xgb, X_lgbm = imputer.fit_transform(X_xgb), imputer.fit_transform(X_lgbm)

# Log-transform target variable
y = np.log1p(y)

# Train-test split
X_train_xgb, X_test_xgb, y_train, y_test = train_test_split(X_xgb, y, test_size=0.2, random_state=42)
X_train_lgbm, X_test_lgbm, _, _ = train_test_split(X_lgbm, y, test_size=0.2, random_state=42)

# Optimize XGBoost with Optuna
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'random_state': 42
    }
    model = XGBRegressor(**params)
    model.fit(X_train_xgb, y_train)
    pred = model.predict(X_test_xgb)
    return mean_absolute_error(y_test, pred)

study_xgb = optuna.create_study(direction='minimize')
study_xgb.optimize(objective_xgb, n_trials=50)
best_params_xgb = study_xgb.best_params
xgb_model = XGBRegressor(**best_params_xgb)
xgb_model.fit(X_train_xgb, y_train)
xgb_pred = np.expm1(xgb_model.predict(X_test_xgb))

# Optimize LightGBM with Optuna
def objective_lgbm(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 50),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'random_state': 42
    }
    model = LGBMRegressor(**params)
    model.fit(X_train_lgbm, y_train)
    pred = model.predict(X_test_lgbm)
    return mean_absolute_error(y_test, pred)

study_lgbm = optuna.create_study(direction='minimize')
study_lgbm.optimize(objective_lgbm, n_trials=50)
best_params_lgbm = study_lgbm.best_params
lgbm_model = LGBMRegressor(**best_params_lgbm)
lgbm_model.fit(X_train_lgbm, y_train)
lgbm_pred = np.expm1(lgbm_model.predict(X_test_lgbm))

# Ensemble predictions (90% XGBoost, 10% LightGBM)
ensemble_pred = (0.9 * xgb_pred + 0.1 * lgbm_pred)

# Evaluate model
print(f"MAE XGBoost: {mean_absolute_error(np.expm1(y_test), xgb_pred):.4f}")
print(f"MAE LightGBM: {mean_absolute_error(np.expm1(y_test), lgbm_pred):.4f}")
print(f"MAE Ensemble (90% XGBoost, 10% LightGBM): {mean_absolute_error(np.expm1(y_test), ensemble_pred):.4f}")


[I 2025-03-18 16:39:25,287] A new study created in memory with name: no-name-bbecc130-1630-4536-904b-0cf9fa7cb157
[I 2025-03-18 16:39:28,660] Trial 0 finished with value: 1.8033489493874109 and parameters: {'n_estimators': 991, 'max_depth': 3, 'learning_rate': 0.18802617935350052, 'subsample': 0.5055348572807952, 'colsample_bytree': 0.5544211474901319}. Best is trial 0 with value: 1.8033489493874109.
[I 2025-03-18 16:39:36,598] Trial 1 finished with value: 1.6770690481408337 and parameters: {'n_estimators': 670, 'max_depth': 10, 'learning_rate': 0.04386714041880845, 'subsample': 0.7823025782026443, 'colsample_bytree': 0.7958185092986652}. Best is trial 1 with value: 1.6770690481408337.
[I 2025-03-18 16:39:39,690] Trial 2 finished with value: 1.6622340763770873 and parameters: {'n_estimators': 968, 'max_depth': 3, 'learning_rate': 0.07442712595122661, 'subsample': 0.5507971879257119, 'colsample_bytree': 0.5294418201943205}. Best is trial 2 with value: 1.6622340763770873.
[I 2025-03-18 1

[LightGBM] [Warning] min_data_in_leaf is set=32, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=32
[LightGBM] [Warning] min_data_in_leaf is set=32, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=32
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004474 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4089
[LightGBM] [Info] Number of data points in the train set: 701, number of used features: 315
[LightGBM] [Info] Start training from score 4.296787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

[I 2025-03-18 16:42:09,518] Trial 0 finished with value: 1.84853920632397 and parameters: {'n_estimators': 587, 'learning_rate': 0.03521530329895833, 'num_leaves': 41, 'min_data_in_leaf': 32, 'max_depth': 6, 'colsample_bytree': 0.7210493430173269}. Best is trial 0 with value: 1.84853920632397.


[LightGBM] [Warning] min_data_in_leaf is set=32, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=32
[LightGBM] [Warning] min_data_in_leaf is set=34, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=34
[LightGBM] [Warning] min_data_in_leaf is set=34, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=34
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003800 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4083
[LightGBM] [Info] Number of data points in the train set: 701, number of used features: 312
[LightGBM] [Info] Start training from score 4.296787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

[I 2025-03-18 16:42:10,960] Trial 1 finished with value: 1.860731409532413 and parameters: {'n_estimators': 973, 'learning_rate': 0.04479817434308457, 'num_leaves': 80, 'min_data_in_leaf': 34, 'max_depth': 9, 'colsample_bytree': 0.8389518435418618}. Best is trial 0 with value: 1.84853920632397.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-03-18 16:42:59,105] Trial 48 finished with value: 1.7244612428503912 and parameters: {'n_estimators': 237, 'learning_rate': 0.0787591897522543, 'num_leaves': 78, 'min_data_in_leaf': 5, 'max_depth': 3, 'colsample_bytree': 0.5676286141728323}. Best is trial 41 with value: 1.6289915906337555.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-03-18 16:42:59,419] Trial 49 finished with value: 1.9341802291560128 and parameters: {'n_estimators': 330, 'learning_rate': 0.09889816719715809, 'num_leaves': 71, 'min_data_in_leaf': 50, 'max_depth': 4, 'colsample_bytree': 0.5856205592098752}. Best is trial 41 with value: 1.6289915906337555.


[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002880 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3956
[LightGBM] [Info] Number of data points in the train set: 701, number of used features: 276
[LightGBM] [Info] Start training from score 4.296787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [